<a href="https://colab.research.google.com/github/salonip/ChatBots/blob/master/A_very_simple_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:

import json
import numpy as np 
import nltk 
import tensorflow as tf
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [80]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [81]:
import os
project_path = "/content/drive/My Drive/chatbot/"
os.chdir(project_path)

In [82]:

with open("intents.json") as file:
    bot = json.load(file)

In [83]:
words = []
labels = []
intent_part_x = []
intent_part_y = []


for intent in bot["intents"]:
    for pattern in intent["patterns"]:
        token = nltk.wordpunct_tokenize(pattern) #Tokenizing the pattern
        #Since token is a list no need of appending we just use the function extend
        words.extend(token)
        intent_part_x.append(token)
        intent_part_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [84]:
len(words)

365

In [85]:
labels[0:10]

['greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'options',
 'Password Reset',
 'E-mail id',
 'Blue Sreen issue',
 'Deleted files',
 'Unsaved Document']

In [86]:
intent_part_x[0:2]

[['Hi', 'there'], ['Hello']]

In [87]:
intent_part_y[0:5]

['greeting', 'greeting', 'greeting', 'greeting', 'greeting']

In [88]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
words = [ps.stem(w.lower()) for w in words if w not in punctuations]
words = sorted(list(set(words)))
labels = sorted(labels)

In [89]:
out_empty = [0 for i,col in enumerate(labels)]
out_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [90]:
train = []
output = []
for x, doc in enumerate(intent_part_x):
    bow = []

    token = [ps.stem(w) for w in doc]

    for w in words:
        if w in token:
            bow.append(1)
        else:
            bow.append(0)


    train.append(bow)  #Creating the Train 
    output.append(labels.index(intent_part_y[x])) #Creating the output list

In [91]:
class_index = dict(zip(labels,range(0,len(labels))))
class_index

{'Blue Sreen issue': 0,
 'Deleted files': 1,
 'E-mail id': 2,
 'Mouse': 3,
 'Password Reset': 4,
 'Printing issues': 5,
 'Slow Computer': 6,
 'Slow Internet': 7,
 'USB': 8,
 'Unexpected Shut Down': 9,
 'Unsaved Document': 10,
 'Vague': 11,
 'Virus': 12,
 'goodbye': 13,
 'greeting': 14,
 'noanswer': 15,
 'options': 16,
 'thanks': 17}

In [92]:
from keras.utils import to_categorical

# one hot encode
y = to_categorical(output)

In [98]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(500, input_shape=(len(train[0]),),activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(class_index),activation='softmax'))


In [99]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [100]:
model.fit(np.array(train), y, epochs=10, batch_size=8, verbose=1)


Epoch 1/10
11/11 [==============================] - 0s 2ms/step - loss: 2.8868 - accuracy: 0.0114
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 2.6664 - accuracy: 0.4886
Epoch 3/10
11/11 [==============================] - 0s 2ms/step - loss: 2.4556 - accuracy: 0.7159
Epoch 4/10
11/11 [==============================] - 0s 2ms/step - loss: 2.2441 - accuracy: 0.7955
Epoch 5/10
11/11 [==============================] - 0s 2ms/step - loss: 2.0320 - accuracy: 0.8295
Epoch 6/10
11/11 [==============================] - 0s 2ms/step - loss: 1.7869 - accuracy: 0.8977
Epoch 7/10
11/11 [==============================] - 0s 2ms/step - loss: 1.5778 - accuracy: 0.9091
Epoch 8/10
11/11 [==============================] - 0s 2ms/step - loss: 1.3576 - accuracy: 0.9318
Epoch 9/10
11/11 [==============================] - 0s 2ms/step - loss: 1.1556 - accuracy: 0.9432
Epoch 10/10
11/11 [==============================] - 0s 2ms/step - loss: 0.9846 - accuracy: 0.9432


In [97]:
from tensorflow.keras.models import load_model
model = load_model('chatbotdemo')

In [101]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 500)               69000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 18)                9018      
Total params: 78,018
Trainable params: 78,018
Non-trainable params: 0
_________________________________________________________________


In [102]:
def bow(sentence,words):
    bag = [ 0 for i in range(len(words))]

    sentence_token = nltk.wordpunct_tokenize(sentence)
    sentence_token = [ps.stem(word.lower()) for word in sentence_token]

    for token in sentence_token:
        for i, word in enumerate(words):
            if word == token:
                bag[i] = 1
    
    return np.array(bag).reshape(-1,len(train[0]))

In [63]:
!pip install flask-ngrok

In [64]:
import tf.keras.backend.tensorflow_backend as tb
tb._SYMBOLIC_SCOPE.value = True

In [103]:
def talk(inp):
    global model
    if inp.lower() == "quit":
        return 
    input_data = [bow(inp, words)]
    results = model.predict(input_data)[0]
    results_index = np.argmax(results)
    tag = labels[results_index]

    if results[results_index] >= 0.05:
	    #Looping through the json file
        for tags in bot["intents"]:
            if tags["tag"] == tag:
                responses = tags["responses"]
        return(np.random.choice(responses))

    else:
        return("I dont quite Understand, Ask another question")


In [106]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return render_template("index.html") 
@app.route("/get")
def get_bot_response():    
    userText = request.args.get('msg')  
    print(userText)
    response = str(talk(userText))
    print(response)  
    return response
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://102ac74b03e4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/Jul/2020 16:46:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2020 16:46:35] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2020 16:46:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/Jul/2020 16:46:55] "GET /get?msg=sing%20me%20a%20poem HTTP/1.1" 200 -


sing me a poem
Offering support for 
1) Password Reset
2) Trouble-Shooting issues
3) Virus Issues
4) Printer issues
5) Other IT tasks


Advancements:

*   Use Embeddings instead of BOW

*  Try BERT *model*

